In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from collections import defaultdict
import copy
import os

In [6]:
from CIFARnet import *

#### This notebook contains a collection of runs on the CIFAR10 dataset
The file <b>'CIFARnet.py'</b> in the same directory contains [pytorch-lightning](https://github.com/PyTorchLightning/pytorch-lightning) modules for CIFARnet (LeNet variation) and ResNet. You can make your own module by inheriting from the <b>'CIFARnetTemplate'</b> class instead of <b>'torch.nn.Module'</b> and providing a mandatory argument <i>hparams</i> (Union[dict, argparse.Namespace]) with specified <i>'learning_rate'</i>, <i>'batch_size'</i> and <i>'weight_decay'</i> attributes. All runs were logged to Weights and Biases platform and a short report with their analysis can be found here: https://app.wandb.ai/culpritgene/cifar10_canonical/reports/CIFAR10-classification-with-ResNet-and-a-simple-convnet.--VmlldzoxMzkwMDA

In [6]:
def configure_CIFARnet():
    ### Configure Convolutional Blocks
    ConvBlockTemplate = {'kernel_size':(3,3),'stride':1, 'padding':1,
                     'pool_kernel_size':(2, 2), 'pool_stride':(2, 2),
                     'dropout_rate':0,
                     'BatchNorm':True,
                     'activation':'relu'}

    Conv2dBlocks = defaultdict(lambda: ConvBlockTemplate.copy())
    Conv2dBlocks['cvb1'].update({'Cin':3, 'Cout':16})
    Conv2dBlocks['cvb2'].update({'Cin':16, 'Cout':32})
    Conv2dBlocks['cvb3'].update({'Cin':32, 'Cout':64, 'dropout_rate':0.3})
    Conv2dBlocks['cvb4'].update({'Cin':64, 'Cout':128,})
    del Conv2dBlocks['cvb4']['pool_kernel_size']
    del Conv2dBlocks['cvb4']['pool_stride']
    ConvBlocksTypes = ['MaxPool', 'MaxPool', 'MaxPool', 'Conv']

    ### Configure FC block
    FcBlock = {'params':(4*4*128, 512, 256, 10),
                      'activations': ('relu','tanh','None'),
                      'dropouts': (0.3, 0.3, 0),
                      'BatchNorm1ds':(True, False, False),
                      'bias':True,
                      'xavier':True}

    hparams = {'Conv2dBlocks':dict(Conv2dBlocks), 'ConvBlocksTypes':ConvBlocksTypes, 'FcBlock': FcBlock}
    hparams['batch_size'] = 32
    hparams['learning_rate'] = 2e-4
    hparams['weight_decay'] = 0
    return hparams

In [7]:
def configure_ResNet(blocks_in_bundle=3, dropout_rate_2d=0, weight_decay=0, batchnorm2d=(True,True)):
    """
        Configures hparams dictionary to be passed to the pytorch-lightning Module
    """
    ### First convolutional Block
    FirstConvBlock = {'Cin':3, 'Cout':16, 'kernel_size':(3,3), 'stride':1, 'padding':1,
                # 'pool_kernel_size':(2, 2), 'pool_stride':(2, 2),
                 'dropout_rate':0,
                 'BatchNorm':True,
                 'activation':'relu'}

    ### Configure Residual Blocks
    ResBlockTemplate = {'kernel_size':(3,3),'stride':1, 'padding':1,
                     'dropout_rate':dropout_rate_2d, 'batchnorm2d':batchnorm2d, 'activation':'relu'}
    
    ### Make a list of layer bundles (inside each bundle all except first (optionally) ResNet blocks share
    ### the same number of input and output channels. Stride=2 may be set for the first Conv2d of the first 
    ### ResNet block in each bundle this way reducing spatial dimentions of the image. 
    BundleTemplate = {'blocks_num':blocks_in_bundle, 'Cin_first':3, 'stride_first':1, 'block_hparams':ResBlockTemplate}
    Conv2dBundles = defaultdict(lambda: copy.deepcopy(BundleTemplate))

    Conv2dBundles['cv_bundle1'].update({'Cin_first':16, 'stride_first':1})
    Conv2dBundles['cv_bundle1']['block_hparams'].update({'Cin':16, 'Cout':16})
    Conv2dBundles['cv_bundle2'].update({'Cin_first':16, 'stride_first':2})
    Conv2dBundles['cv_bundle2']['block_hparams'].update({'Cin':32, 'Cout':32})
    Conv2dBundles['cv_bundle3'].update({'Cin_first':32, 'stride_first':2})
    Conv2dBundles['cv_bundle3']['block_hparams'].update({'Cin':64, 'Cout':64})
    
    ### Configure global hyperparameters
    hparams = {'Conv2dBundles':dict(Conv2dBundles), 
               'FirstConvBlock':FirstConvBlock, 'FirstMaxPool':False,
               'FcBlock': (64,10), 'out_of_conv_dim':(8,8)}
    hparams['batch_size'] = 32
    hparams['learning_rate'] = 2e-4
    hparams['weight_decay'] = weight_decay
    return hparams

In [8]:
dropouts = [0.25, 0.2, 0.15, 0.1, 0.07, 0.03, 0.01, 0.0]
weight_decays = [5e-3, 1e-3, 5e-4, 1e-5]

In [5]:
from pytorch_lightning.callbacks import LearningRateLogger, EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
path = '/home/moonstrider/Neural_Networks_Experiments/VISION/CIFAR10/'
early_stopping = EarlyStopping('val_loss', min_delta=0.0, patience=3)
tmpdir = '/home/moonstrider/Neural_Networks_Experiments/VISION/CIFAR10/chkpt'

In [10]:
Model_configs = {}
Model_configs['CIFARnet'] = configure_CIFARnet()
for d in dropouts:
    Model_configs[f'ResNet-drop-{d}-wd-0'] = configure_ResNet(dropout_rate_2d=d)
for wd in weight_decays:
    Model_configs[f'ResNet-drop-0-wd-{wd}'] = configure_ResNet(weight_decay=wd)

In [48]:
def set_trainer_and_logger(run_name, project='CIFAR10_canonical', offline=True, tmpdir=tmpdir):
    wandb_logger = WandbLogger(name=run_name, save_dir=path+'WandB_Logs/', offline=offline, project=project)
    model_chekpoint = ModelCheckpoint(os.path.join(tmpdir, f'{run_name}'+'canonical_run-{epoch}-{val_loss:.2f}'), 
                                  verbose=True, period=3, save_top_k=2)
    trainer = pl.Trainer(gpus=1, max_epochs=22, logger=wandb_logger, callbacks=[early_stopping], checkpoint_callback=None)
    return trainer, model_chekpoint

#### Running ResNet18 from torchvision

In [45]:
from torchvision.models import resnet18
class ResNetFromTorchvision(CIFARnetTemplate):
    def __init__(self, hparams, model_cls):
        super(ResNetFromTorchvision, self).__init__()
        self.hparams = hparams
        self.model = model_cls()
    
    def forward(self, x):
        return self.model.forward(x)

In [46]:
ResNet18_ImageNet_config = defaultdict(lambda x: 0)
ResNet18_ImageNet_config['batch_size'] = 32
ResNet18_ImageNet_config['learning_rate'] = 2e-4
ResNet18_ImageNet_config['weight_decay'] = 0

In [47]:
run_name = 'ResNet18_ImageNet'
trainer, model_chekpoint = set_trainer_and_logger(run_name)
M = ResNetFromTorchvision(dict(ResNet18_ImageNet_config), resnet18)
trainer.fit(M)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


### Running CifarNet and the sweep for Regularization parameters on ResNet20

In [29]:
for i, run_name in enumerate(Model_configs):
    trainer, model_chekpoint = set_trainer_and_logger(run_name)
    M = CIFARnet(Model_configs[run_name]) if i==0 else ResNetCifar(Model_configs[run_name])
    print('Starting training for 22 epochs with early stopping...\n')
    trainer.fit(M)
    print('Training is finished.\n')
    print('Launching on test dataset...\n')
    trainer.test(M)
print('All runs finished!')

### Comparing ResNet110 training with and without BatchNorm2d

<font color='Red'><b>Without BatchNorm:

In [10]:
ResNet110_conf = configure_ResNet(blocks_in_bundle=18, dropout_rate_2d=0.1, 
                                  batchnorm2d=(False, False), 
                                  weight_decay=5e-3)

In [12]:
run_name = 'ResNet110_no_bn'
trainer, model_chekpoint = set_trainer_and_logger(run_name)
M = ResNetCifar(ResNet110_conf)
trainer.fit(M)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


Configuring model ResNet110_no_bn...

Starting training for 22 epochs with early stopping...



<font color='Green'><b>With BatchNorm:

In [18]:
ResNet110_conf = configure_ResNet(blocks_in_bundle=18, dropout_rate_2d=0.1, 
                                  batchnorm2d=(True, True), 
                                  weight_decay=5e-3)

In [31]:
run_name = 'ResNet110_with_bn'
trainer, model_chekpoint = set_trainer_and_logger(run_name)
M = ResNetCifar(ResNet110_conf)
trainer.fit(M)

### Adding augmentation and a bit of experiments with bundles of ResNet blocks configuration

In [42]:
FirstConvBlock = {'Cin':3, 'Cout':16, 'kernel_size':(3,3), 'stride':1, 'padding':1,
            # 'pool_kernel_size':(2, 2), 'pool_stride':(2, 2),
             'dropout_rate':0,
             'BatchNorm':True,
             'activation':'relu'}

### Configure Residual Blocks
ResBlockTemplate = {'kernel_size':(3,3),'stride':1, 'padding':1,
                 'dropout_rate':0.15, 'batchnorm2d':(True,True), 'activation':'relu'}

### Make a list of layer bundles (inside each bundle all except first (optionally) ResNet blocks share
### the same number of input and output channels. Stride=2 may be set for the first Conv2d of the first 
### ResNet block in each bundle this way reducing spatial dimentions of the image. 
BundleTemplate = {'blocks_num':3, 'Cin_first':3, 'stride_first':1, 'block_hparams':ResBlockTemplate}
Conv2dBundles = defaultdict(lambda: copy.deepcopy(BundleTemplate))

Conv2dBundles['cv_bundle1'].update({'blocks_num':5, 'Cin_first':16, 'stride_first':1})
Conv2dBundles['cv_bundle1']['block_hparams'].update({'Cin':32, 'Cout':32})
Conv2dBundles['cv_bundle2'].update({'blocks_num':7, 'Cin_first':32, 'stride_first':2})
Conv2dBundles['cv_bundle2']['block_hparams'].update({'Cin':64, 'Cout':64})
Conv2dBundles['cv_bundle3'].update({'blocks_num':7, 'Cin_first':64, 'stride_first':2})
Conv2dBundles['cv_bundle3']['block_hparams'].update({'Cin':128, 'Cout':128})
Conv2dBundles['cv_bundle4'].update({'blocks_num':5, 'Cin_first':128, 'stride_first':2})
Conv2dBundles['cv_bundle4']['block_hparams'].update({'Cin':256, 'Cout':256})

### Configure global hyperparameters
hparams = {'Conv2dBundles':dict(Conv2dBundles), 
           'FirstConvBlock':FirstConvBlock, 'FirstMaxPool':False,
           'FcBlock': (256,10), 'out_of_conv_dim':(4,4)}
hparams['batch_size'] = 32
hparams['learning_rate'] = 2e-4
hparams['weight_decay'] = 5e-3

In [43]:
run_name = 'MyResNet-vII'
trainer, model_chekpoint = set_trainer_and_logger(run_name)
M = ResNetCifar(hparams)
print('Starting training for 25 epochs with early stopping...\n')

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


Starting training for 25 epochs with early stopping...



<font color='Purple'><b>Adding augmentations:

In [12]:
M.prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [13]:
from torchvision import transforms
M.train_dataset.dataset.transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                    transforms.RandomGrayscale(),
                    transforms.RandomPerspective(),
                    transforms.RandomRotation(degrees=45),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

In [14]:
trainer.fit(M)

Files already downloaded and verified
Files already downloaded and verified


wandb: Wandb version 0.9.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

    | Name                  | Type        | Params
--------------------------------------------------
0   | FirstConv             | ConvBlock   | 480   
1   | FirstConv.Conv2d      | Conv2d      | 448   
2   | FirstConv.BatchNorm2d | BatchNorm2d | 32    
3   | cv_bundle1            | Sequential  | 88 K  
4   | cv_bundle1.0          | ResNetBlock | 14 K  
5   | cv_bundle1.0.conv1    | Conv2d      | 4 K   
6   | cv_bundle1.0.conv2    | Conv2d      | 9 K   
7   | cv_bundle1.0.bn1      | BatchNorm2d | 64    
8   | cv_bundle1.0.bn2      | BatchNorm2d | 64    
9   | cv_bundle1.0.dropout1 | Dropout2d   | 0     
10  | cv_bundle1.0.dropout2 | Dropout2d   | 0     
11  | cv_bundle1.1          | ResNetBlock | 18 K  
12  | cv_bundle1.1.conv1    | Conv2d      | 9 K   
13  | cv_bundle1.1.conv2    | Conv2d      | 9 K   
14  | cv_bundle1.1.bn1      | BatchNorm2d | 64    
15  | cv_bundle1.1.bn2      

wandb: WARNING Adding to old History rows isn't currently supported.  Step 0 < 1; dropping {'train_loss': 4.11558723449707, 'train_acc': 0.15625, 'epoch': 0}.


1